<a href="https://colab.research.google.com/github/smannan/LIFXBulbAnalysis/blob/main/JoinPGEAndBulbData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
def drop_seconds_from_timestamp(df):
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M')
  df['timestamp'] = pd.to_datetime(df['timestamp'])

In [51]:
def agg_pge_hourly(df):
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H')
  df['timestamp'] = pd.to_datetime(df['timestamp'])

  df = df.groupby(['timestamp']).agg({
    'cost': 'sum',
    'usage': 'sum'
  })

  return df.reset_index()

In [52]:
bulb_file = '/content/drive/Shared drives/CMPE295B/FinalBulbData/2021-08-26_13_2021-09-25_00_hourly.csv'
bulb_data = pd.read_csv(bulb_file, infer_datetime_format = True).drop(['Unnamed: 0'], axis=1, inplace=False)
bulb_data['timestamp'] = pd.to_datetime(bulb_data['timestamp'])

In [53]:
pge_file = '/content/drive/Shared drives/CMPE295B/ProcessedPGEData/pge_data_from_influx_2021-08-26-00_15_2021-09-21-00_00.csv'
pge_data = pd.read_csv(pge_file, infer_datetime_format = True).drop(['Unnamed: 0'], axis=1, inplace=False)
pge_hourly = agg_pge_hourly(pge_data)

In [54]:
print (len(pge_hourly))
pge_hourly.head()

625


,timestamp,cost,usage
0,2021-08-26 00:00:00,0.06,0.24
1,2021-08-26 01:00:00,0.08,0.29
2,2021-08-26 02:00:00,0.08,0.28
3,2021-08-26 03:00:00,0.08,0.28
4,2021-08-26 04:00:00,0.08,0.28


In [55]:
print (len(bulb_data))
bulb_data.head()

708


,timestamp,brightness,kelvin,kWH
0,2021-08-26 13:00:00,1.0,2700.0,0.119
1,2021-08-26 14:00:00,1.0,2700.0,0.510
2,2021-08-26 15:00:00,1.0,2700.0,0.510
3,2021-08-26 16:00:00,1.0,2700.0,0.510
4,2021-08-26 17:00:00,1.0,2700.0,0.510


In [56]:
merged = pd.merge_asof(bulb_data, pge_hourly, on='timestamp')

In [57]:
print (len(merged))
merged.head()

708


,timestamp,brightness,kelvin,kWH,cost,usage
0,2021-08-26 13:00:00,1.0,2700.0,0.119,0.09,0.35
1,2021-08-26 14:00:00,1.0,2700.0,0.510,0.11,0.38
2,2021-08-26 15:00:00,1.0,2700.0,0.510,0.09,0.34
3,2021-08-26 16:00:00,1.0,2700.0,0.510,0.08,0.29
4,2021-08-26 17:00:00,1.0,2700.0,0.510,0.11,0.40


In [59]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 708 entries, 0 to 707
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   708 non-null    datetime64[ns]
 1   brightness  708 non-null    float64       
 2   kelvin      708 non-null    float64       
 3   kWH         708 non-null    float64       
 4   cost        708 non-null    float64       
 5   usage       708 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 38.7 KB
